In [ ]:
import time
from datetime import date, datetime, timedelta
import hashlib
import requests
import hmac
from urllib.parse import urlencode
import os

import pandas as pd
import matplotlib.pyplot as plot

KEY = ""
SECRET = ""
BASE_URL = "https://api.binance.com"  # production base url
# BASE_URL = 'https://testnet.binance.vision' # testnet base url


def get_timestamp():
    return int(time.time() * 1000)

def to_datetime(timestamp):
    return datetime.fromtimestamp(timestamp / 1000)

def to_timestamp(datetime):
    return int(datetime.timestamp() * 1000)

def send_signed_request(http_method, url_path, payload={}):
    query_string = urlencode(payload, True)
    if query_string:
        query_string = "{}&timestamp={}".format(query_string, get_timestamp())
    else:
        query_string = "timestamp={}".format(get_timestamp())

    url = (
        BASE_URL + url_path + "?" + query_string + "&signature=" + hashing(query_string)
    )
    #print("{} {}".format(http_method, url))
    params = {"url": url, "params": {}}
    response = dispatch_request(http_method)(**params)
    return response.json()

def send_public_request(url_path, payload={}):
    query_string = urlencode(payload, True)
    url = BASE_URL + url_path
    if query_string:
        url = url + "?" + query_string
    #print("{}".format(url))
    response = dispatch_request("GET")(url=url)
    return response.json()

def dispatch_request(http_method):
    session = requests.Session()
    session.headers.update(
        {"Content-Type": "application/json;charset=utf-8", "X-MBX-APIKEY": KEY}
    )
    return {
        "GET": session.get,
        "DELETE": session.delete,
        "PUT": session.put,
        "POST": session.post,
    }.get(http_method, "GET")

def get_day_price(ticker, datetime):
    timestamp = to_timestamp(datetime)
    response = send_public_request("/api/v3/klines", 
                               {"symbol": ticker,
                                "startTime": timestamp,
                                "endTime": timestamp+2*24*3600*1000,
                                "interval": "1d",
                                "limit": 1
                               })
    if len(response) == 0:
        return float('NaN')
    if "code" in response:
        raise Exception("Error(", ticker,"): ", response)
    return float(response[0][4]) # open_time, open, high, low, close, volume, ...


In [ ]:
# tax methods

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

def show_table(df):
    filtered_df = df
    fig = go.Figure(data=[go.Table(
        header={ "values":list(filtered_df.columns),
                    "fill_color":'paleturquoise',
                    "align":'left'},
        cells={"values":filtered_df.transpose().values.tolist(),
                "fill_color":'lavender',
                "align":'left'})
    ])
    fig.show()


df = pd.read_csv("data/2023-binanceTransactionHistory.csv")

targetCoin = "USDT"
resultdf = pd.DataFrame([], columns=[ 'Asset', 'Quantity', targetCoin  ])

assets = ["BTC", "ADA", "AXS", "ETH", "SOL", "DOT"]
assets = df["Coin"].unique()
for asset in assets:
    ticker = asset
    coinPair = ticker + targetCoin
    if ticker == targetCoin:
        price = 1
    else:
        price = get_day_price(coinPair, datetime.now() - timedelta(days = 1))

    localdf = df[df['Coin'].str.contains(ticker) ]
    localdf = localdf[localdf['Operation'].str.contains("Reward")]
    localdf["EUR"] = localdf[localdf['Coin'].str.contains(ticker)]["Change"].apply(lambda x: x * price)
    total = localdf['Change'].sum()
    total_in_target = total * price
    resultdf.loc[len(resultdf.index)] = [ ticker, total, total_in_target ]
    #print(resultdf)
    #show_table(localdf)
    #print("Total ", asset, "in ", targetCoin,"(",df['Change'].sum()," x ", price,"):",total)

fig = px.bar(resultdf, x='Asset', y=targetCoin , hover_data={targetCoin :':.3r'})
fig.show()
show_table(resultdf)
show_table(df)
